<a href="https://colab.research.google.com/github/ea-Mitsuoka/ionicons/blob/master/bqml_xgboost_churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
```
Copyright 2019 Google LLC

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
```
---

# 概要
このノートブックでは、BigQuery MLのXGBoostで解約予測モデル作成を行っています。


### 費用

このチュートリアルでは、Google BigQuery および BigQuery ML を使います。金額について詳細は  [BigQuery ML pricing](https://cloud.google.com/bigquery-ml/pricing) [Pricing
Calculator](https://cloud.google.com/products/calculator/)
を参照ください。

### 事前準備

**このノートブックを実行するには事前に次の手順が必要です。**

1. [GCP プロジェクトの作成もしくは選択](https://console.cloud.google.com/cloud-resource-manager). 最初にアカウントを作成すると、300ドルの無料クレジットが付与されます。

2. [課金設定を有効にします](https://cloud.google.com/billing/docs/how-to/modify-project)

3.   [BigQuery API](https://console.cloud.google.com/flows/enableapi?apiid=bigquery) を有効にします。

4. 下のセルを実行し、Google アカウントでログインして表示されるキーをセルに入力します。

5. 「プロジェクト変数の設定」のセルにご自身のプロジェクトIDを入力してください。

In [ ]:
import pandas as pd
import numpy as np
import sys
from google.cloud import bigquery
from google.colab import auth
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
auth.authenticate_user()

In [ ]:
#@title プロジェクト変数の設定 { run: "auto", display-mode: "form" }
project_id = '' #@param {type:"string"}
#dataset_name = "ga" #@param {type:"string"}
client = bigquery.Client(project=project_id)

まず、モデルを保存するためのデータセットが必要になります。 （エラーが発生した場合は、BigQueryコンソールから"iris"というデータセットを作成してください）。

In [ ]:
%%bash -s "$project_id"

gcloud config set project $1
bq --location=US mk -d churn

Process is terminated.


## データセット読み取り
このセクションでは、Kaggleから読み込んだtelcoデータセットをクエリしています。

In [ ]:
query = '''
SELECT * from churn.telco
'''
df = client.query(query).to_dataframe()

In [ ]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,9732-OUYRN,Female,0,True,False,49,True,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,False,Credit card (automatic),19.0,918.7,False
1,0661-KQHNK,Female,0,True,True,6,True,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,False,Credit card (automatic),19.0,105.5,False
2,4709-LKHYG,Female,0,True,True,29,True,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,False,Electronic check,20.0,540.05,False
3,9824-QCJPK,Male,0,True,False,36,True,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,False,Mailed check,20.0,666.75,False
4,4716-MRVEN,Female,0,False,False,29,True,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,False,Mailed check,20.0,599.3,False


トレーニング データを確認できたので、 BQML で Churn 列を予測する XGBoost モデルを作成します。
Churn 列は True か False で構成されているため、分類モデルでモデル作成を行います。

In [ ]:
query = '''
CREATE OR REPLACE MODEL churn.xgb_ltv
OPTIONS(input_label_cols=['Churn'], 
        model_type='boosted_tree_classifier')
AS
SELECT *  FROM churn.telco
'''
client.query(query)

モデルを作成したら、予測を試します。

In [ ]:
query = """
SELECT customerID,predicted_Churn FROM ML.PREDICT(MODEL churn.xgb_ltv, TABLE churn.telco)
  """
df = client.query(query).to_dataframe()
df

,customerID,predicted_Churn
0,9732-OUYRN,False
1,0661-KQHNK,False
2,4709-LKHYG,False
3,9824-QCJPK,False
4,4716-MRVEN,False
...,...,...
7038,5883-GTGVD,True
7039,5502-RLUYV,True
7040,3001-UNBTL,True
7041,5760-IFJOZ,True


### クリーンアップ
すべての作業が完了したら、以下のURLを参考に、プロジェクトを必要に応じてクリーンアップしてください。
https://cloud.google.com/go/getting-started/delete-tutorial-resources?hl=ja